# Imports and Data loading

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import anndata as ad
#import celltypist
import anndata as ad
from scipy import sparse
from anndata import AnnData
import sys  
#import scib
sys.path.insert(0, '/home/icb/leonie.pohl/masterpraktikum_fibrosis_atlas/masterpraktikum_fibrosis_atlas/source')
#sys.path.append("Desktop/Masterpraktikum/masterpraktikum_fibrosis_atlas/source/utils.py")
#import utils

In [2]:
galapagos_bleo = sc.read("/lustre/groups/ml01/workspace/daniel.michaela.masterpraktikum23/galapagos/galapagos_bleo_pp_qc_anno.h5ad")

In [3]:
galapagos_rad = sc.read("/lustre/groups/ml01/workspace/daniel.michaela.masterpraktikum23/galapagos/adata_radiation_processed.h5ad")

In [4]:
galapagos_rad.obs = pd.read_csv("/lustre/groups/ml01/workspace/daniel.michaela.masterpraktikum23/galapagos/radiation_obs_annotation2.csv")

In [3]:
publics = sc.read("/lustre/groups/ml01/workspace/daniel.michaela.masterpraktikum23/public_data/merged_data.h5ad")

In [14]:
publics

AnnData object with n_obs × n_vars = 87871 × 23767
    obs: 'author_annotation', 'scDblFinder_score', 'scDblFinder_class', 'manual_celltype_annotation', 'batch', 'condition', 'doublet_score', 'predicted_doublet', 'timepoint', 'author_annotation_coarse', 'dataset', 'fibrotic/control', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'low_hierarchy', 'high_hierarchy', 'harmonized_anno', 'coarse_harmonized_anno', 'low_hierarchy_fine', 'high_hierarchy_fine', 'low_hierarchy_coarse', 'high_hierarchy_coarse'
    var: 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches

# Reduce adatas --> only relevant info for merging

which information should anndatas contain:

adata.X --> raw counts
in adata.obs

- "batch" --> number or name identifying batch/sample --> SAMP_subject_ID / "tube_ID"
- "author_annotation" --> annotation given by paper --> CELL_celltype_compartment (only for bleo)
- "manual_celltype_annotation" --> your annotation  --> manual_celltype_annotation_fine_leiden03 / annotation
- "condition" --> "healthy" or "bleo" or other treatment for fibrosis --> SAMP_condition
-  Phase
- stage/ time --> SAMP_stage
- SC-Prep groups --> SC_prep

In [7]:
galapagos_rad.X = galapagos_rad.layers["logcounts"]

In [8]:
galapagos_rad.obs["batch"] = galapagos_rad.obs["SAMP_subject_ID"]
galapagos_rad.obs["manual_celltype_annotation"] = galapagos_rad.obs["annotation"]
galapagos_rad.obs["condition"] = galapagos_rad.obs["SAMP_condition"]
galapagos_rad.obs["timepoint"] = galapagos_rad.obs["SAMP_stage"]

In [9]:
galapagos_bleo.obs["batch"] = galapagos_bleo.obs["SAMP_subject_ID"]
galapagos_bleo.obs["manual_celltype_annotation"] = galapagos_bleo.obs["manual_celltype_annotation_fine_leiden03"]
galapagos_bleo.obs["author_annotation"] = galapagos_bleo.obs["CELL_celltype_compartment"]

galapagos_bleo.obs["condition"] = galapagos_bleo.obs["SAMP_condition"]
galapagos_bleo.obs["timepoint"] = galapagos_bleo.obs["SAMP_stage"]



In [10]:
del galapagos_rad.layers
del galapagos_rad.obsp
del galapagos_rad.varm
del galapagos_rad.obsm
del galapagos_rad.uns
del galapagos_rad.var

In [11]:
del galapagos_bleo.layers
del galapagos_bleo.obsp
del galapagos_bleo.obsm
del galapagos_bleo.uns
del galapagos_bleo.var

In [12]:
galapagos_rad.obs = galapagos_rad.obs[['batch', "manual_celltype_annotation", "condition","Phase", "SAMP_stage", "SC_prep" ]]

In [13]:
galapagos_bleo.obs = galapagos_bleo.obs[['batch','author_annotation', "manual_celltype_annotation", "condition","Phase", "SAMP_stage", "SC_prep" ]]

In [14]:
galapagos_rad.obs["batch"] = galapagos_rad.obs["batch"].astype("category")
galapagos_rad.obs["manual_celltype_annotation"] = galapagos_rad.obs["manual_celltype_annotation"].astype("category")
galapagos_rad.obs["condition"] = galapagos_rad.obs["condition"].astype("category")
galapagos_rad.obs["Phase"] = galapagos_rad.obs["Phase"].astype("category")
galapagos_rad.obs["SAMP_stage"] = galapagos_rad.obs["SAMP_stage"].astype("category")
galapagos_rad.obs["SC_prep"] = galapagos_rad.obs["SC_prep"].astype("category")

In [15]:
galapagos_bleo.obs["batch"] = galapagos_bleo.obs["batch"].astype("category")

In [16]:
galapagos_rad.obs.index

RangeIndex(start=0, stop=343071, step=1)

In [17]:
galapagos_rad.obs.index = galapagos_rad.obs.index.astype(str)

In [18]:
galapagos_rad.obs.index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '343061', '343062', '343063', '343064', '343065', '343066', '343067',
       '343068', '343069', '343070'],
      dtype='object', length=343071)

In [19]:
galapagos_rad.obs.dtypes

batch                         category
manual_celltype_annotation    category
condition                     category
Phase                         category
SAMP_stage                    category
SC_prep                       category
dtype: object

In [20]:
del publics.obsp
del publics.layers
del publics.varm
del publics.uns
del publics.obsm
del publics.var

In [21]:
publics.obs["manual_celltype_annotation"] = publics.obs["harmonized_anno"] 

In [22]:
publics.obs = publics.obs[["author_annotation", "manual_celltype_annotation", "batch", "condition", "timepoint", "dataset", "fibrotic/control", "coarse_harmonized_anno"]]

In [23]:
publics

AnnData object with n_obs × n_vars = 87871 × 23767
    obs: 'author_annotation', 'manual_celltype_annotation', 'batch', 'condition', 'timepoint', 'dataset', 'fibrotic/control', 'coarse_harmonized_anno'

# Merging 

In [33]:
adatas = {
    "galapagos_bleo": galapagos_bleo,
    "galapagos_rad": galapagos_rad
 }
galapagos_adata = ad.concat(adatas, label="dataset", index_unique="_", join="outer", axis = 0)#.obs

In [34]:
galapagos_adata.obs["condition"] = galapagos_adata.obs["condition"].replace(["BLM"], "bleomycin") # 'RAD', 'SHAM'] ['BLM', 'SHAM']
galapagos_adata.obs["condition"] = galapagos_adata.obs["condition"].replace(["RAD"], "radiation") # 'RAD', 'SHAM'] ['BLM', 'SHAM']
galapagos_adata.obs["condition"] = galapagos_adata.obs["condition"].replace(["SHAM"], "untreated")

In [35]:
galapagos_adata.obs["fibrotic/control"] = galapagos_adata.obs["condition"].apply(lambda x: "control" if x == "untreated" else "fibrotic")


In [44]:
galapagos_adata.obs["batch"] = galapagos_adata.obs.apply(lambda row: str(row["batch"]) + "_" + str(row["dataset"]), axis=1)


In [45]:
galapagos_adata.obs

,batch,author_annotation,manual_celltype_annotation,condition,Phase,SAMP_stage,SC_prep,dataset,fibrotic/control
1_Sal_veh_d21_AAACCCAAGACATATG-1_galapagos_bleo,1.0_galapagos_bleo,lymphoid,CD4 T-cells,untreated,S,day21,group1,galapagos_bleo,control
1_Sal_veh_d21_AAACCCAGTATTCTCT-1_galapagos_bleo,1.0_galapagos_bleo,lymphoid,B-cells,untreated,S,day21,group1,galapagos_bleo,control
1_Sal_veh_d21_AAACCCATCCGCTGTT-1_galapagos_bleo,1.0_galapagos_bleo,mononuclear_phagocytes,Dendritic cells,untreated,G1,day21,group1,galapagos_bleo,control
1_Sal_veh_d21_AAACGCTAGCGGGTTA-1_galapagos_bleo,1.0_galapagos_bleo,lymphoid,B-cells,untreated,S,day21,group1,galapagos_bleo,control
1_Sal_veh_d21_AAAGGATGTCCGGTCA-1_galapagos_bleo,1.0_galapagos_bleo,lymphoid,CD4 T-cells,untreated,G2M,day21,group1,galapagos_bleo,control
...,...,...,...,...,...,...,...,...,...
343066_galapagos_rad,74.0_galapagos_rad,NaN,Alveolar macrophages,radiation,S,19weeks,day3_group9,galapagos_rad,fibrotic
343067_galapagos_rad,74.0_galapagos_rad,NaN,AT2,radiation,G1,19weeks,day3_group9,galapagos_rad,fibrotic
343068_galapagos_rad,74.0_galapagos_rad,NaN,B-cells,radiation,G2M,19weeks,day3_group9,galapagos_rad,fibrotic
343069_galapagos_rad,74.0_galapagos_rad,NaN,B-cells,radiation,G2M,19weeks,day3_group9,galapagos_rad,fibrotic


In [46]:
adatas = {
    "public": publics ,
    "galapagos": galapagos_adata
 }
adata = ad.concat(adatas, join="outer", axis = 0)#.obs

In [47]:
adata.obs

,author_annotation,manual_celltype_annotation,batch,condition,timepoint,dataset,fibrotic/control,coarse_harmonized_anno,Phase,SAMP_stage,SC_prep
AAACCTGAGGACATTA-1_xie,Col13a1,Myofibroblasts,1_xie,untreated,NaN,xie,control,Myofibroblasts,NaN,NaN,NaN
AAACCTGCAGTCGTGC-1_xie,Myofibroblasts,Peribronchial fibroblasts,1_xie,untreated,NaN,xie,control,Fibroblasts,NaN,NaN,NaN
AAACCTGCATGAAGTA-1_xie,Lipofibroblasts,Alveolar macrophages,1_xie,untreated,NaN,xie,control,Alveolar macrophages,NaN,NaN,NaN
AAACCTGTCTCGCATC-1_xie,Myofibroblasts,Peribronchial fibroblasts,1_xie,untreated,NaN,xie,control,Fibroblasts,NaN,NaN,NaN
AAACGGGTCCTAGAAC-1_xie,Myofibroblasts,Peribronchial fibroblasts,1_xie,untreated,NaN,xie,control,Fibroblasts,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
343066_galapagos_rad,NaN,Alveolar macrophages,74.0_galapagos_rad,radiation,NaN,galapagos_rad,fibrotic,NaN,S,19weeks,day3_group9
343067_galapagos_rad,NaN,AT2,74.0_galapagos_rad,radiation,NaN,galapagos_rad,fibrotic,NaN,G1,19weeks,day3_group9
343068_galapagos_rad,NaN,B-cells,74.0_galapagos_rad,radiation,NaN,galapagos_rad,fibrotic,NaN,G2M,19weeks,day3_group9
343069_galapagos_rad,NaN,B-cells,74.0_galapagos_rad,radiation,NaN,galapagos_rad,fibrotic,NaN,G2M,19weeks,day3_group9


In [48]:
adata.obs["batch"] = adata.obs["batch"].astype("category")

In [49]:
adata.write("/lustre/groups/ml01/workspace/daniel.michaela.masterpraktikum23/all_datasets_merged/complete_merged_public_and_galapagos.h5ad")